## Check GPU

In [2]:
!nvidia-smi

Sat Jan  3 20:19:15 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   25C    P8             26W /  300W |     759MiB /  16303MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/home/faruk/Documents/nvidia-da/poker-card/train


## Install YOLO11 via Ultralytics

In [4]:
%pip install "ultralytics<=8.3.40" supervision roboflow wandb

import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 15847MiB)
Setup complete ✅ (16 CPUs, 46.9 GB RAM, 416.7/915.3 GB disk)


## Optional Settings

In [5]:
import wandb

# Disable ultralytics tracking
!yolo settings sync=False

# Enable W&B logging in Ultralytics  #NEW
!yolo settings wandb=True

JSONDict("/home/faruk/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/faruk/Documents/faruk/datasets",
  "weights_dir": "/home/faruk/Documents/faruk/realtime-poker-card-detection-jetson/weights",
  "runs_dir": "/home/faruk/Documents/faruk/realtime-poker-card-detection-jetson/runs",
  "uuid": "55a89c66d590b4f5e4a272dd743bb229894610632fe0a6c7524c5a09383b0903",
  "sync": false,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": true,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
JSONDict("/home/faruk/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/faruk/Documents/faruk/datasets",
  "weights_dir": "/home/faruk/Documents/faruk/realtime-poker-card-detection-jetson/weights",
 

In [6]:
import wandb
wandb.login()

wandb: Currently logged in as: mdfaruk-aet (mdfaruk-aet-sylhet-agricultural-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Fine-tune YOLO11 on custom dataset

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/ks-fsm9o/pelvis-ap-x-ray) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

 - Poker Cards Dataset: https://universe.roboflow.com/roboflow-jvuqo/poker-cards-fmjio/dataset/4

In [7]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/home/faruk/Documents/nvidia-da/poker-card/train/datasets


In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="jNNh2gmG3E4aDKPlvoAp")
project = rf.workspace("augmented-startups").project("playing-cards-ow27d")
version = project.version(4)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Playing-Cards-4 in yolov11:: 100%|██████████| 48478/48478 [00:05<00:00, 9392.16it/s] 


## Custom Training

In [9]:
%cd {HOME}

/home/faruk/Documents/nvidia-da/poker-card/train


In [10]:

!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=120 imgsz=640 plots=True project=poker-cards-detection name=run1

100%|██████████████████████████████████████| 18.4M/18.4M [00:10<00:00, 1.84MB/s]
New https://pypi.org/project/ultralytics/8.3.246 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 15847MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/faruk/Documents/nvidia-da/poker-card/train/datasets/Playing-Cards-4/data.yaml, epochs=120, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=poker-cards-detection, name=run1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=Tru

## Thanks 

Original notebook from: https://github.com/roboflow/notebooks